In [ ]:
import time
start_time = time.time()
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark import SparkContext
from pyspark.sql.functions import * #for SQL functions
from pyspark.sql.window import Window #for window function
from datetime import datetime

In [ ]:
"""Cluster Specifications
5 Nodes
Driver: I3.xlarge
runtime version: 10.3 (includes Apache Spark 3.2.1, Scala 2.12)
30.5 GB Memory, 4 cores
Min workers-2
Max workers-5
"""

In [ ]:
spark = SparkSession.builder \
            .appName("dds") \
            .getOrCreate()

# Add configuration for accessing S3

In [ ]:
aws_access_key = '####'
aws_secret_key = '#####'
spark._jsc.hadoopConfiguration().set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.1') 
spark._jsc.hadoopConfiguration().set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", aws_access_key)
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", aws_secret_key)
spark._jsc.hadoopConfiguration().set('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1')
spark._jsc.hadoopConfiguration().set('spark.network.timeout','7200s')
spark._jsc.hadoopConfiguration().set('spark.executor.heartbeatInterval','1200s')

# Data Pre-processing:

In [ ]:
def IntegerSafe(value): # In case there are non-integer type to be converted.
    try:
        return int(value)
    except:
        return None

def FloatSafe(value): # In case there are non-integer type to be converted.
    try:
        return float(value)
    except:
        return None
    
def toTimeSafe(inval):
    inval = inval.strip("\"") # Timestamp starting and ending with a double quotation mark.
    try:
        return datetime.strptime(inval, "%Y/%m/%d")
    except ValueError:
        return None
    
def BoolSafe(value): # In case there are non-integer type to be converted.
    try:
        if value =="True": return True
        else: return False
    except:
        return None

## 1. Covid data

In [ ]:
# read covid file using pyspark context
header = sc.textFile("s3://msds697distdata/COVID-19_Cases_by_Geography_and_Date.csv").first()

covid_file = sc.textFile("s3://msds697distdata/COVID-19_Cases_by_Geography_and_Date.csv")\
             .filter(lambda x: x!= header)\
             .map(lambda x : x.split(','))\
             .map(lambda x : (toTimeSafe(x[0])
                              , x[1]
                              , x[2]
                              , FloatSafe(x[3])
                              , FloatSafe(x[4])
                              , FloatSafe(x[5])
                              , FloatSafe(x[6])                                            
                            ))

schema = StructType([ StructField("Specimen_Collection_Date", DateType(), True),
                      StructField("area_type", StringType(), True),
                      StructField("id", StringType(), True),
                      StructField("acs_population", FloatType(), True),
                      StructField("New_Confirmed_Cases", FloatType(), True),
                      StructField("Cumulative_Confirmed_Cases", FloatType(), True),
                      StructField("Rate_of_Cumulative_Confirmed_Case", FloatType(), True)    
                    ])

covid_df = spark.createDataFrame(covid_file, schema)

In [ ]:
covid_df.show(10)

+------------------------+--------------------+--------------------+--------------+-------------------+--------------------------+---------------------------------+
Specimen_Collection_Date| area_type| id|acs_population|New_Confirmed_Cases|Cumulative_Confirmed_Cases|Rate_of_Cumulative_Confirmed_Case|
+------------------------+--------------------+--------------------+--------------+-------------------+--------------------------+---------------------------------+
 2020-03-03|Analysis Neighbor...|Bayview Hunters P...| 37917.0| 0.0| 0.0| null|
 2020-03-03|Analysis Neighbor...| Bernal Heights| 25167.0| 0.0| 0.0| null|
 2020-03-03|Analysis Neighbor...| Castro/Upper Market| 22502.0| 0.0| 0.0| null|
 2020-03-03|Analysis Neighbor...| Chinatown| 14438.0| 0.0| 0.0| null|
 2020-03-03|Analysis Neighbor...| Excelsior| 39879.0| null| null| null|
 2020-03-03|Analysis Neighbor...|Financial Distric...| 21537.0| 0.0| 0.0| null|
 2020-03-03|Analysis Neighbor...| Glen Park| 8651.0| 0.0| 0.0| null|
 2020-03-03|Analysis Neighbor...| Golden Gate Park| 63.0| null| null| null|
 2020-03-03|Analysis Neighbor...| Haight Ashbury| 19275.0| 0.0| 0.0| null|
 2020-03-03|Analysis Neighbor...| Hayes Valley| 19711.0| 0.0| 0.0| null|
+------------------------+--------------------+--------------------+--------------+-------------------+--------------------------+---------------------------------+
only showing top 10 rows

In [ ]:
# Filtering to only required columns - neighbourhood, covid counts and population
population = covid_df.filter(covid_df['area_type'] == 'Analysis Neighborhood')\
            .select('id', 'acs_population')


In [ ]:
# aggregating to find covid counts in each neighborhood for each month from March 2020
covid_df = covid_df.filter(covid_df['area_type'] == 'Analysis Neighborhood')\
    .groupBy('Specimen_Collection_Date','id')\
    .agg(sum('New_Confirmed_Cases').alias('daily_new_cases'))\
    .select('Specimen_Collection_Date','id',year("Specimen_Collection_Date").alias('year'), month("Specimen_Collection_Date").alias('month'), 'daily_new_cases')\
    .sort('Specimen_Collection_Date')\
    .groupBy('year','month','id')\
    .agg(sum('daily_new_cases').alias('monthly_cases'))\
    .sort(['id', 'year', 'month'])\
    .join(population, 'id', 'left_outer')\
    .select('id', 'year', 'month', 'monthly_cases', 'acs_population')\
    .distinct()

In [ ]:
# calculating covid case %age in each neighborhood in each month
covid_df = covid_df.select('id', 'year', 'month','monthly_cases', 'acs_population', round((covid_df['monthly_cases']/covid_df['acs_population']) * 100, 2).alias(
'percent_population_with_covid')).sort(['year', 'month','id'])

In [ ]:
#calculating total number of monthly covid cases across SF
total_covid_cases_per_month = covid_df.groupBy('year', 'month').agg(sum('monthly_cases').alias('total_cases'))

## 2. Business data

In [ ]:
schema = StructType([ StructField("location_id", StringType(), True),
                      StructField("bus_accnt_num", StringType(), False),
                      StructField("Owner_name", StringType(), True),
                      StructField("DBA_name", StringType(), True),
                      StructField("address", StringType(), True),
                      StructField("city", StringType(), True),
                      StructField("state", StringType(), True),
                     StructField("zipcode", StringType(), True),
                     StructField("bus_start_date", DateType(), True),
                     StructField("bus_end_date", DateType(), True),
                     StructField("loc_start_date", DateType(), True),
                     StructField("loc_end_date", DateType(), True),
#                      StructField("mail_address", StringType(), True),
#                      StructField("mail_city", StringType(), True),
#                      StructField("mail_zipcode", StringType(), True),
#                      StructField("mail_state", StringType(), True),
                     StructField("NAICS_code", StringType(), True),
                     StructField("NAICS_code_desc", StringType(), True),
                     StructField("parking_tax", StringType(), True),
                     StructField("transient_occ_tax", StringType(), True),
#                      StructField("LIC_code", StringType(), True),
#                      StructField("LIC_code_desc", StringType(), True),
                     StructField("supervisor_dist", StringType(), True),
                     StructField("neigh_analysis", StringType(), True),
                     StructField("bus_corr", StringType(), True),
                     StructField("bus_loc", StringType(), True),
                     StructField("uniqueID", StringType(), True),
                     StructField("SF_neighbourhoods", StringType(), True),
                     StructField("Police_district", StringType(), True),
                     StructField("current_super_dist", StringType(), True),
                     StructField("analysis_neighbor", StringType(), True),
                     StructField("neighborhoods", StringType(), True)
                    ])

In [ ]:
#Reading the SF business data
business = sc.textFile("s3://msds697distdata/Registered_Business_Locations_-_San_Francisco.tsv").map(lambda x : x.split('\t'))
def DateSafe(value):
    try:
        return datetime.strptime(value, '%m/%d/%Y')
    except:
        return None
    
    
head = business.first()
print(business.count())
business = business.filter(lambda x: x != head)\
                   .map(lambda x:(x[0], x[1], x[2], x[3], x[4], x[5], x[6], IntegerSafe(x[7]),\
                          DateSafe(x[8]),DateSafe(x[9]), DateSafe(x[10]), DateSafe(x[11]),\
#                         x[12], x[13],IntSafe(x[14]), x[15], 
                          x[16], x[17], BoolSafe(x[18]), BoolSafe(x[19]), x[22], x[23], x[24], x[25], x[26],\
                          IntegerSafe(x[27]),IntegerSafe(x[28]),IntegerSafe(x[29]), IntegerSafe(x[30]),IntegerSafe(x[31]))) 

business.count()
business_df = spark.createDataFrame(business, schema)

286179

In [ ]:
# Removing missing neighborhoods as they are outside San Francisco
business_df = business_df.filter("neigh_analysis != '' and state == 'CA'").drop('state', 'city')

In [ ]:
# Removing duplicates on the basis of owner name, DBA name, address and business start and end dates
dropbusDF = business_df.dropDuplicates(["Owner_name","DBA_name","bus_start_date","address","bus_end_date"])
print("Distinct count of department & salary : "+str(dropbusDF.count()))

Distinct count of department & salary : 232341

In [ ]:
dropbusDF.show(10)

+--------------+-------------+--------------------+--------------------+--------------------+-------+--------------+------------+--------------+------------+--------------------+--------------------+-----------+-----------------+---------------+--------------------+--------+--------------------+--------------------+-----------------+---------------+------------------+-----------------+-------------+
 location_id|bus_accnt_num| Owner_name| DBA_name| address|zipcode|bus_start_date|bus_end_date|loc_start_date|loc_end_date| NAICS_code| NAICS_code_desc|parking_tax|transient_occ_tax|supervisor_dist| neigh_analysis|bus_corr| bus_loc| uniqueID|SF_neighbourhoods|Police_district|current_super_dist|analysis_neighbor|neighborhoods|
+--------------+-------------+--------------------+--------------------+--------------------+-------+--------------+------------+--------------+------------+--------------------+--------------------+-----------+-----------------+---------------+--------------------+--------+--------------------+--------------------+-----------------+---------------+------------------+-----------------+-------------+
1259359-09-201| 1117215| !High Frog!| !High Frog!|1750 Sacramento S...| 94109| 2020-09-04| null| 2020-09-04| null| 7100-7199 7220-7229| Multiple| false| false| 3| Nob Hill| |POINT (-122.42171...|1259359-09-201-11...| 105| 4| 3| 21| 105|
1121809-09-161| 1058627|"Elizabeth ""Liz"...|"Elizabeth ""Liz"...|225 Hartford St U...| 94114| 2016-09-20| 2019-01-01| 2016-09-20| 2019-01-01| 6100-6299|Private Education...| false| false| 8| Castro/Upper Market| |POINT (-122.43367...|1121809-09-161-10...| 115| 3| 5| 5| 115|
0430837-01-001| 0430837|"Fasioen-Delbeek ...| Studio In Bloom| 456 4th Ave| 94118| 2008-07-23| 2014-06-30| 2008-07-23| 2014-06-30| | | false| false| 1| Inner Richmond| |POINT (-122.46173...|0430837-01-001-04...| 5| 8| 4| 11| 5|
1294919-01-221| 1130970|"John Henry ""Man...|"John Henry ""Man...| 309 Holloway Ave| 94112| 2021-09-01| null| 2021-09-01| null| | | false| false| 11|Oceanview/Merced/...| |POINT (-122.45632...|1294919-01-221-11...| 64| 10| 1| 24| 64|
0480634-01-001| 0480634| "Pawz ""R"" Us Llc"| Pawz R Us Llc| 755 Green St 2e| 94133| 2013-06-15| 2018-06-30| 2013-06-15| 2018-06-30| | | false| false| 3| Chinatown| |POINT (-122.41160...|0480634-01-001-04...| 107| 6| 3| 6| 107|
0480634-02-001| 0480634| "Pawz ""R"" Us Llc"| The Wizard Of Pawz| 755 Green St 2e| 94133| 2013-06-15| 2018-06-30| 2013-06-15| 2018-06-30| | | false| false| 3| Chinatown| |POINT (-122.41160...|0480634-02-001-04...| 107| 6| 3| 6| 107|
1289684-10-211| 1128968|"S.m.e ""Samoends...|"S.m.e ""Samoends...| 26 Northridge Rd| 94124| 2021-10-15| null| 2021-10-15| null| 4400-4599| Retail Trade| false| false| 10|Bayview Hunters P...| |POINT (-122.37491...|1289684-10-211-11...| 78| 2| 9| 1| 78|
1293533-12-211| 1130442|"The ""Perfect"" ...|"The ""Perfect"" ...|735 Taylor St Apt...| 94108| 2021-12-15| null| 2021-12-15| null|5600-5699 7100-71...| Multiple| false| false| 3| Nob Hill| |POINT (-122.41234...|1293533-12-211-11...| 50| 6| 3| 21| 50|
0393138-01-001| 0393138| #10 Sports Llc| #10 Sports Llc|444 Townsend St ...| 94107| 2005-10-31| 2014-09-01| 2005-10-31| 2014-09-01| | | false| false| 6| Mission Bay| |POINT (-122.39824...|0393138-01-001-03...| 32| 1| 10| 4| 32|
1208914-01-191| 1096501| &Open Gifts Inc| &Open Gifts Inc| 1161 Mission St| 94103| 2017-07-01| null| 2017-07-01| null| 4400-4599| Retail Trade| false| false| 6| South of Market| |POINT (-122.41222...|1208914-01-191-10...| 32| 1| 10| 34| 32|
+--------------+-------------+--------------------+--------------------+--------------------+-------+--------------+------------+--------------+------------+--------------------+--------------------+-----------+-----------------+---------------+--------------------+--------+--------------------+--------------------+-----------------+---------------+------------------+-----------------+-------------+
only showing top 10 rows

##  3.Crime data

In [ ]:
#Reading SF crime data
crime = sc.textFile("s3://msds697distdata/Police_Department_Incident_Reports__2018_to_Present.csv").map(lambda x : x.split(','))

columns = crime.first()
crime = crime.filter(lambda x: x!=columns)

In [ ]:

def TimeSafe(inval):
    inval = inval.strip("\'") # Timestamp starting and ending with a double quotation mark.
    try:
        return datetime.strptime(inval, "%Y/%m/%d")
    except ValueError:
        return None
    
    
crime = crime.map(lambda x:[TimeSafe(x[1]),
                            x[14],
                            x[21],
                            FloatSafe(x[23]),
                            FloatSafe(x[24]),
                            x[25]
                            ])


schema_crime = StructType([
                          StructField('Incident Date',DateType(),True),
                          StructField('Incident Category',StringType(),True),
                          StructField('Analysis Neighborhood',StringType(),True),
                          StructField('Latitude',FloatType(),True),
                          StructField('Longitude',FloatType(),True),
                         StructField('Point',StringType(),True)])
                          

df_crime = spark.createDataFrame(crime, schema_crime)

In [ ]:
df_crime.show(10)

+-------------+------------------+---------------------+--------+-----------+--------------------+
Incident Date| Incident Category|Analysis Neighborhood|Latitude| Longitude| Point|
+-------------+------------------+---------------------+--------+-----------+--------------------+
 2021-09-29|Malicious Mischief| Ingleside| null| null| |
 2021-05-14| Arson| Bayview Hunters P...|37.74426|-122.387375|POINT (-122.38737...|
 2021-03-01| Lost Property| | null| null| |
 2021-04-01| Lost Property| | null| null| |
 2021-06-01| Larceny Theft| | null| null| |
 2021-09-22| Larceny Theft| | null| null| |
 2021-09-17| Larceny Theft| | null| null| |
 2021-05-12| Larceny Theft| | null| null| |
 2021-05-09| Larceny Theft| | null| null| |
 2021-05-11| Larceny Theft| | null| null| |
+-------------+------------------+---------------------+--------+-----------+--------------------+
only showing top 10 rows

In [ ]:
# convert date into year and Month
df_crime = df_crime.withColumn('year',year(df_crime['Incident Date']))
df_crime = df_crime.withColumn('month',month(df_crime['Incident Date']))
df_crime = df_crime.withColumnRenamed('Incident Category','IncidentCategory')

df_crime = df_crime.withColumnRenamed('Incident Category','IncidentCategory')
df_crime = df_crime.withColumnRenamed('Analysis Neighborhood','AnalysisNeighborhood')

# Filterout the vehicle recovery from the date

df_crime = df_crime.filter("IncidentCategory !='Recovered Vehicle'")

# Split crime incident into larceny theft and others Larceny Theft 
df_crime = df_crime.withColumn('Larceny_theft',df_crime['IncidentCategory']=='Larceny Theft')
df_crime = df_crime.withColumn('other_incident',df_crime['IncidentCategory']!='Larceny Theft')

# Filter out incidents without addressinfo/latitude neighborhood
df_crime = df_crime.filter("Latitude IS NOT NULL")

In [ ]:
df_crime.orderBy('year',ascending=0).show(3)

+-------------+------------------+--------------------+---------+----------+--------------------+----+-----+-------------+--------------+
Incident Date| IncidentCategory|AnalysisNeighborhood| Latitude| Longitude| Point|year|month|Larceny_theft|other_incident|
+-------------+------------------+--------------------+---------+----------+--------------------+----+-----+-------------+--------------+
 2022-01-01| Robbery| Northern| 2.0| 37.803493| -122.44527037912071|2022| 1| false| true|
 2022-01-03|Malicious Mischief| Taraval| 4.0| 37.74673| -122.477023892666|2022| 1| false| true|
 2022-01-03| Drug Offense| South of Market|37.778084|-122.40615|POINT (-122.40615...|2022| 1| false| true|
+-------------+------------------+--------------------+---------+----------+--------------------+----+-----+-------------+--------------+
only showing top 3 rows

In [ ]:
# Groupby Neighbothood month and year
df_crime_grouped = df_crime.groupBy([df_crime['AnalysisNeighborhood'],df_crime.year,df_crime.month]).count().orderBy('year',ascending=0)

In [ ]:
df_crime_grouped.show(10)

+--------------------+----+-----+-----+
AnalysisNeighborhood|year|month|count|
+--------------------+----+-----+-----+
 Hayes Valley|2022| 1| 54|
 Central|2022| 1| 221|
 Southern|2022| 2| 20|
 North Beach|2022| 2| 1|
 Open or Active|2022| 1| 59|
 Lone Mountain/USF|2022| 1| 23|
 Visitacion Valley|2022| 1| 34|
 Portola|2022| 2| 2|
 Glen Park|2022| 1| 13|
 Bernal Heights|2022| 1| 30|
+--------------------+----+-----+-----+
only showing top 10 rows

# Joining datasets

In [ ]:
#Rows in each dataset
print(dropbusDF.distinct().count(),covid_df.distinct().count(), df_crime_grouped.distinct().count())

232341 984 2612

In [ ]:
#joining crime data to business data
joined = dropbusDF.join(df_crime_grouped, df_crime_grouped.AnalysisNeighborhood== dropbusDF.neigh_analysis, 'left_outer')

In [ ]:
#joining the covid data to the above joined data
joined_covid = joined.join(covid_df,(covid_df.id== joined.neigh_analysis) & (covid_df.year== joined.year) & (covid_df.month== joined.month), 'left_outer').cache()

In [ ]:
joined_covid.count()

Out[26]: 11589390

In [ ]:
#dropping double variables
joined_df = joined_covid.drop(covid_df.year).drop(covid_df.month).drop('id')

In [ ]:
#filtering the start and end dates of the business as per our requirement
#The data has been made monthly time series for each business as per the neighborhood
def year_month(value):
    try:
        if value < 10:
            return "0"+ str(value)
        return value
    except:
        return None
month_udf = udf(year_month)
# Keeping records with start date before the monthly counter date 
# Keeping accounts with end date either null or end date after 2018 as our data is from 2018
# Keeping records with business close date either null or before the monthly counter date

final_df = joined_df.withColumn('year_month', concat(col('year'),month_udf(col('month'))))\
            .withColumn('year_month_strt', concat(year('loc_start_date'),month_udf(month('loc_start_date'))))\
            .withColumn('year_month_end', concat(year('loc_end_date'),month_udf(month('loc_end_date'))))\
            .filter("year_month_strt <= year_month")\
            .filter("year_month_end IS NULL or year_month_end >= 2018")\
            .filter("year_month_end IS NULL or year_month_end <= year_month").drop("year").drop("month").cache()

In [ ]:
final_df.show(2)

+--------------+-------------+--------------------+--------------------+-----------------+-------+--------------+------------+--------------+------------+----------+---------------+-----------+-----------------+---------------+--------------------+--------+--------------------+--------------------+-----------------+---------------+------------------+-----------------+-------------+--------------------+-----+-------------+--------------+-----------------------------+----------+---------------+--------------+
 location_id|bus_accnt_num| Owner_name| DBA_name| address|zipcode|bus_start_date|bus_end_date|loc_start_date|loc_end_date|NAICS_code|NAICS_code_desc|parking_tax|transient_occ_tax|supervisor_dist| neigh_analysis|bus_corr| bus_loc| uniqueID|SF_neighbourhoods|Police_district|current_super_dist|analysis_neighbor|neighborhoods|AnalysisNeighborhood|count|monthly_cases|acs_population|percent_population_with_covid|year_month|year_month_strt|year_month_end|
+--------------+-------------+--------------------+--------------------+-----------------+-------+--------------+------------+--------------+------------+----------+---------------+-----------+-----------------+---------------+--------------------+--------+--------------------+--------------------+-----------------+---------------+------------------+-----------------+-------------+--------------------+-----+-------------+--------------+-----------------------------+----------+---------------+--------------+
0467656-01-001| 0467656| 3gear Systems Inc| 3gear Systems Inc| 164 S Park St| 94107| 2012-03-05| 2014-11-26| 2012-03-05| 2014-11-26| | | false| false| 6|Financial Distric...| |POINT (-122.39471...|0467656-01-001-04...| 31| 1| 10| 8| 31|Financial Distric...| 286| null| null| null| 201901| 201203| 201411|
1014153-12-141| 0119543|Abm Parking Services|Ampco System Parking|1 Embarcadero Ctr| 94111| 1979-05-01| 2013-12-31| 2014-01-01| 2013-12-31| | | false| false| 3|Financial Distric...| |POINT (-122.39648...|1014153-12-141-01...| 108| 6| 3| 8| 108|Financial Distric...| 286| null| null| null| 201901| 201401| 201312|
+--------------+-------------+--------------------+--------------------+-----------------+-------+--------------+------------+--------------+------------+----------+---------------+-----------+-----------------+---------------+--------------------+--------+--------------------+--------------------+-----------------+---------------+------------------+-----------------+-------------+--------------------+-----+-------------+--------------+-----------------------------+----------+---------------+--------------+
only showing top 2 rows

In [ ]:
final_df.count()

Out[30]: 9431693

In [ ]:
# Creating a business closure flag during covid

df_closed_flag = final_df.withColumn( 'covid_close_flag_new', when((final_df.year_month_end == final_df.year_month), 1).otherwise(0))


In [ ]:
df_closed_flag.groupBy('covid_close_flag_new').count().show()

+--------------------+-------+
covid_close_flag_new| count|
+--------------------+-------+
 1| 69334|
 0|9362359|
+--------------------+-------+

In [ ]:
#Missing Value imputation for covid data

df_miss_val = df_closed_flag.withColumn("monthly_cases", when((df_closed_flag.monthly_cases.isNull()), 0).otherwise(df_closed_flag.monthly_cases))\
                            .withColumn("acs_population", when((df_closed_flag.acs_population.isNull()), 0).otherwise(df_closed_flag.acs_population))\
                            .withColumn("percent_population_with_covid", when((df_closed_flag.percent_population_with_covid.isNull()), 0).otherwise(df_closed_flag.percent_population_with_covid))

In [ ]:
df_miss_val.show(3)

+--------------+-------------+--------------------+--------------------+-----------------+-------+--------------+------------+--------------+------------+----------+----------------+-----------+-----------------+---------------+--------------------+--------+--------------------+--------------------+-----------------+---------------+------------------+-----------------+-------------+--------------------+-----+-------------+--------------+-----------------------------+----------+---------------+--------------+--------------------+
 location_id|bus_accnt_num| Owner_name| DBA_name| address|zipcode|bus_start_date|bus_end_date|loc_start_date|loc_end_date|NAICS_code| NAICS_code_desc|parking_tax|transient_occ_tax|supervisor_dist| neigh_analysis|bus_corr| bus_loc| uniqueID|SF_neighbourhoods|Police_district|current_super_dist|analysis_neighbor|neighborhoods|AnalysisNeighborhood|count|monthly_cases|acs_population|percent_population_with_covid|year_month|year_month_strt|year_month_end|covid_close_flag_new|
+--------------+-------------+--------------------+--------------------+-----------------+-------+--------------+------------+--------------+------------+----------+----------------+-----------+-----------------+---------------+--------------------+--------+--------------------+--------------------+-----------------+---------------+------------------+-----------------+-------------+--------------------+-----+-------------+--------------+-----------------------------+----------+---------------+--------------+--------------------+
0467656-01-001| 0467656| 3gear Systems Inc| 3gear Systems Inc| 164 S Park St| 94107| 2012-03-05| 2014-11-26| 2012-03-05| 2014-11-26| | | false| false| 6|Financial Distric...| |POINT (-122.39471...|0467656-01-001-04...| 31| 1| 10| 8| 31|Financial Distric...| 286| 0.0| 0.0| 0.0| 201901| 201203| 201411| 0|
1014153-12-141| 0119543|Abm Parking Services|Ampco System Parking|1 Embarcadero Ctr| 94111| 1979-05-01| 2013-12-31| 2014-01-01| 2013-12-31| | | false| false| 3|Financial Distric...| |POINT (-122.39648...|1014153-12-141-01...| 108| 6| 3| 8| 108|Financial Distric...| 286| 0.0| 0.0| 0.0| 201901| 201401| 201312| 0|
0335856-51-001| 0335856|Ace Parking Manag...| Ace Parking Iii Llc| 201 Spear St| null| 2000-02-01| null| 2014-05-01| null| 8100-8139|Certain Services| false| false| 6|Financial Distric...| |POINT (-122.39242...|0335856-51-001-03...| 108| 1| 10| 8| 108|Financial Distric...| 286| 0.0| 0.0| 0.0| 201901| 201405| null| 0|
+--------------+-------------+--------------------+--------------------+-----------------+-------+--------------+------------+--------------+------------+----------+----------------+-----------+-----------------+---------------+--------------------+--------+--------------------+--------------------+-----------------+---------------+------------------+-----------------+-------------+--------------------+-----+-------------+--------------+-----------------------------+----------+---------------+--------------+--------------------+
only showing top 3 rows

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 415.87884736061096 seconds ---

# Connect to MongoDB
## Store aggregates in the database and re-read for machine learning later

In [ ]:
database = '###'
collection = '###'
user_name = '###'
password = '###'
address = 'group-11.akldt.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
# connection_string = f"mongodb+srv://msds_9:<datasystems>@group-11.akldt.mongodb.net/Group-11"

In [ ]:
df_miss_val.write.format("mongo").option("uri",connection_string).mode("append").save()

In [ ]:
df = spark.read.format("mongo").option("uri",connection_string).load()

In [ ]:
df.show()

+--------------------+--------------------+----------+--------------------+--------------------+---------------+-----------------+--------------------+--------------+--------------------+-----------------+-------------+--------+-------------------+--------------------+-------------------+-----+--------------------+------------------+-------------------+-------------------+--------------+-------------+-------------------+-------------+-----------+-----------------------------+---------------+-----------------+--------------------+----------+--------------+---------------+-------+
AnalysisNeighborhood| DBA_name|NAICS_code| NAICS_code_desc| Owner_name|Police_district|SF_neighbourhoods| _id|acs_population| address|analysis_neighbor|bus_accnt_num|bus_corr| bus_end_date| bus_loc| bus_start_date|count|covid_close_flag_new|current_super_dist| loc_end_date| loc_start_date| location_id|monthly_cases| neigh_analysis|neighborhoods|parking_tax|percent_population_with_covid|supervisor_dist|transient_occ_tax| uniqueID|year_month|year_month_end|year_month_strt|zipcode|
+--------------------+--------------------+----------+--------------------+--------------------+---------------+-----------------+--------------------+--------------+--------------------+-----------------+-------------+--------+-------------------+--------------------+-------------------+-----+--------------------+------------------+-------------------+-------------------+--------------+-------------+-------------------+-------------+-----------+-----------------------------+---------------+-----------------+--------------------+----------+--------------+---------------+-------+
 Castro/Upper Market| 2555 Market St Apts| 5300-5399|Real Estate and R...| Bsw Corporation| 3| 38|{621bbdfb28d1927b...| 0.0| 2555 Market St| 5| 0374748| | null|POINT (-122.43678...|1996-10-31 00:00:00| 127| 0| 5| null|1996-10-31 00:00:00|0374748-01-001| 0.0|Castro/Upper Market| 38| false| 0.0| 8| false|0374748-01-001-03...| 201810| null| 199610| null|
 Castro/Upper Market|Constant Fitness,...| 6100-6299|Private Education...|Constant Fitness,...| 3| 38|{621bbdfb28d1927b...| 0.0|2201 Market St Ste C| 5| 1028501| | null|POINT (-122.43104...|2015-10-27 00:00:00| 127| 0| 5|2016-04-30 00:00:00|2015-10-27 00:00:00|1058079-03-161| 0.0|Castro/Upper Market| 38| false| 0.0| 8| false|1058079-03-161-10...| 201810| 201604| 201510| 94114|
 Castro/Upper Market| Pegus Property| 5300-5399|Real Estate and R...|Dotterweich Micha...| 3| 115|{621bbdfb28d1927b...| 0.0| 183 Eureka St| 5| 1012117| | null|POINT (-122.43811...|2015-04-06 00:00:00| 127| 0| 5| null|2015-04-06 00:00:00|1025046-04-151| 0.0|Castro/Upper Market| 115| false| 0.0| 8| false|1025046-04-151-10...| 201810| null| 201504| 94114|
 Castro/Upper Market| Maru Dojo| 7100-7199|Arts, Entertainme...| Feliciano Raymond R| 7| 28|{621bbdfb28d1927b...| 0.0| 736 14th St| 5| 0460757| | null|POINT (-122.42974...|2013-04-01 00:00:00| 127| 0| 5| null|2013-04-01 00:00:00|0460757-03-001| 0.0|Castro/Upper Market| 28| false| 0.0| 8| false|0460757-03-001-04...| 201810| null| 201304| null|
 Castro/Upper Market| Lynn Friedman| 7100-7199|Arts, Entertainme...| Friedman Lynn| 3| 38|{621bbdfb28d1927b...| 0.0|584 Castro St Uni...| 5| 1024391| | null|POINT (-122.43491...|2016-01-16 00:00:00| 127| 1| 5|2018-10-25 00:00:00|2017-06-01 00:00:00|1149995-06-171| 0.0|Castro/Upper Market| 38| false| 0.0| 8| false|1149995-06-171-10...| 201810| 201810| 201706| null|
 Chinatown| 737 Pine I1, Llc| 5300-5399|Real Estate and R...| 737 Pine L1 Lp| 6| 104|{621bbdfb28d1927b...| 14438.0|600 California St 20| 6| 1018390| | null|POINT (-122.40453...|2014-12-15 00:00:00| 78| 0| 3|2020-01-30 00:00:00|2014-12-15 00:00:00|1039582-09-151| 3.0| Chinatown| 104| false| 0.02| 3| false|1039582-09-151-10...| 202105| 202001| 201412| 94108|
 Chinatown| 916 Pacific I5, Llc| 5300-5399|Real Estate and R...| 916 Pacific I5, Llc| 6| 107|{621bbdfb28d1927b...| 14438.0| 916 Pacific Ave| 6| 1066396| | null|POINT (-122.41033...|201